In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Load data for each source

First we load the data from each source and inspect how it looks.

In [2]:
df_source1 = pd.read_csv("./data/source_one.csv")
df_source1 = df_source1.fillna('')  # normalize NaN values
print("Number of records in source 1:", len(df_source1))
df_source1.head(5)

Number of records in source 1: 84


,ID,name,address,phone
0,a1,Westerns Country Inn,"447 West Route 77, Johnscity, AZ 86033, United States of America",928-345-4567
1,a2,Natpolin Hostel,"Kanuk Ayasofya Mahallesi Yeğen Sok. No:3/1 Sultanahmet , Fatih, 34110 Istanbul, Turkey",
2,a3,Sleep In! Cairo Hostel,"11 B,Ramsis St. Marouf Tower.Downtown, 99999 Cairo, Egypt",
3,a4,Mekking House,"11 Lorong Carnarvon, 10220 George Town, Malaysia",
4,a5,Rolando & Chateaux Palazzo Seneca,"Via Cesare Battisti 5, 06044 Isla, Italy",0712 567834


In [3]:
df_source2 = pd.read_csv("./data/source_two.csv")
df_source2 = df_source2.fillna('')  # normalize NaN values
print("Number of records in source 2:", len(df_source2))
df_source2.head(5)

Number of records in source 2: 84


,ID,name,city,country,zip,street,phone
0,ba1,Westerns Country Inn Bed & Breakfast,Johnscity,United States,86033,West Route 77 447,+1 928-345-4567
1,ba2,Natpolin Hotel,Istanbul,Turkey,34110,"Kanuk Ayasofya Mahallesi Yeğen Sok. No:3/1 Sultanahmet , Fatih, ,",
2,ba3,Sleep IN Cairo Hostel,Cairo,Egypt,11511,"11 B, Ramses Street, Downtown",
3,ba4,Mekking House,George Town,Malaysia,10220,11 Lorong Carnarvon,
4,ba5,Palazzo Seneca,Isla,Italy,6046,"Via Cesare Battisti, 5",+39 0712 567834


### Normalize record identifiers

In this particular example, source one has the following columns: name, address and phone. Source two has name, city, country, zip, street and phone. 

In order to compare each record identifier separately, we need to make sure that the two datasets have the same columns. In this case, we have two options: 

* in the second source, combine each single address identifier per record into one address column
* in the first source, programmatically extract city, country, zip and street from the single address column

The second option will allow us to be more precise in our comparisons and will also help us create better record candidate pairs. Fortunately, there are tools in the Python ecosystem that will allow us to do such data pre-preprocessing.

We start with extracting specific address information from the first source by using a Python package called "postal" (https://github.com/openvenues/pypostal).

In [4]:
from postal.parser import parse_address

In [5]:
def split_address_column(address_column):
    """
    This function splits a single address string into city, country, zip code and street by using the postal.parse_address function.
    """
    result = parse_address(address_column)
    street_elements = []
    
    street = ""
    city = ""
    zipcode = ""
    country = ""
    
    for element in result:
        if element[1] == "city":
            city = element[0]
        elif element[1] == "postcode":
            zipcode = element[0]
        elif element[1] == "country":
            country = element[0]
        else:
            street_elements.append(element[0])
            
    street = " ".join(street_elements)
    
    return city, country, zipcode, street

In [6]:
df_source1[['city','country', "zip", "street"]] = df_source1.address.apply(lambda x: pd.Series(split_address_column(x)))
del df_source1["address"]

With the above commands we have split the address column from source 1  into city, country, zipcode and street. The pstal Python library has also did some internal pre-processing of the strings, which is unfortunately not easily reversable without modifying the library itself, however, we will use similar pre-processing on the data from source 2 later on.

In [7]:
df_source1

,ID,name,phone,city,country,zip,street
0,a1,Westerns Country Inn,928-345-4567,johnscity,united states of america,86033,447 west route 77 az
1,a2,Natpolin Hostel,,istanbul,turkey,34110,kanuk ayasofya mahallesi yeğen sok. no 3/1 sultanahmet fatih
2,a3,Sleep In! Cairo Hostel,,cairo,egypt,99999,11 b ramsis st. marouf tower.downtown
3,a4,Mekking House,,george town,malaysia,,11 lorong carnarvon 10220
4,a5,Rolando & Chateaux Palazzo Seneca,0712 567834,isla,italy,06044,via cesare battisti 5
...,...,...,...,...,...,...,...
79,a495,OldCity Open-hotel - Suites & Apartments,,cairo,egypt,99999,39 chark compound campus new cairo city
80,a496,Live Party Bed & Breakfast,,desenzano del garda,italy,25015,woal cavour 2
81,a497,Lala Hostala,,chefchaouen,morocco,91000,lalla lambda 22
82,a498,Orka Mountain villa 10,,ovacik,turkey,48340,


# Data Pre-processing

We still have multiple inconsistencies between data sources in terms of how each column is repsented. If investigate each column in more detail we can see that:
* the country names are inconsistent, for example, in one source we can see "United States", while in the other "United States of America"
* phone number in source 1 have no country code, while the records in source 2 have the country code

In [8]:
# lowercase all text fields
df_source1=df_source1.apply(lambda x: x.astype(str).str.lower() if(x.dtype == 'object') else x)
df_source2=df_source2.apply(lambda x: x.astype(str).str.lower() if(x.dtype == 'object') else x)

In [9]:
df_source2

,ID,name,city,country,zip,street,phone
0,ba1,westerns country inn bed & breakfast,johnscity,united states,86033,west route 77 447,+1 928-345-4567
1,ba2,natpolin hotel,istanbul,turkey,34110,"kanuk ayasofya mahallesi yeğen sok. no:3/1 sultanahmet , fatih, ,",
2,ba3,sleep in cairo hostel,cairo,egypt,11511,"11 b, ramses street, downtown",
3,ba4,mekking house,george town,malaysia,10220,11 lorong carnarvon,
4,ba5,palazzo seneca,isla,italy,6046,"via cesare battisti, 5",+39 0712 567834
...,...,...,...,...,...,...,...
79,ba495,newcity suites & apartments,cairo,egypt,11835,23 arban compound infront of new aac campus- new cairo,
80,ba496,bed & breakfast party house,desenzano del garda,italy,25015,via woal verdi 10,+39 44 33 443
81,ba497,hostal la lambada,chefchaouene,morocco,91000,rue sidi ahme,+212 44-33445
82,ba498,orka my villa,istanbul,turkey,48345,"ovacık mahallesi, istanbul caddesi",


In [10]:
df_source1

,ID,name,phone,city,country,zip,street
0,a1,westerns country inn,928-345-4567,johnscity,united states of america,86033,447 west route 77 az
1,a2,natpolin hostel,,istanbul,turkey,34110,kanuk ayasofya mahallesi yeğen sok. no 3/1 sultanahmet fatih
2,a3,sleep in! cairo hostel,,cairo,egypt,99999,11 b ramsis st. marouf tower.downtown
3,a4,mekking house,,george town,malaysia,,11 lorong carnarvon 10220
4,a5,rolando & chateaux palazzo seneca,0712 567834,isla,italy,06044,via cesare battisti 5
...,...,...,...,...,...,...,...
79,a495,oldcity open-hotel - suites & apartments,,cairo,egypt,99999,39 chark compound campus new cairo city
80,a496,live party bed & breakfast,,desenzano del garda,italy,25015,woal cavour 2
81,a497,lala hostala,,chefchaouen,morocco,91000,lalla lambda 22
82,a498,orka mountain villa 10,,ovacik,turkey,48340,


Next we will solve the problem of inconsistent country names by using a Python library called pycountry (https://pypi.org/project/pycountry/)

In [11]:
import pycountry

In [12]:
def normalize_country_to_alpha2_code(country_name):
    """
    This function will receive a country name in an arbitrary format and will return a 2 letter country code for that country as per the alpha_2 country code format.
    This is achieved by using the pycountry Python library.
    """
    country_alpha2 = pycountry.countries.search_fuzzy(country_name)[0].alpha_2
    return country_alpha2

In [13]:
df_source1["country"] = df_source1.country.apply(lambda x: normalize_country_to_alpha2_code(x))

In [14]:
df_source1

,ID,name,phone,city,country,zip,street
0,a1,westerns country inn,928-345-4567,johnscity,US,86033,447 west route 77 az
1,a2,natpolin hostel,,istanbul,TR,34110,kanuk ayasofya mahallesi yeğen sok. no 3/1 sultanahmet fatih
2,a3,sleep in! cairo hostel,,cairo,EG,99999,11 b ramsis st. marouf tower.downtown
3,a4,mekking house,,george town,MY,,11 lorong carnarvon 10220
4,a5,rolando & chateaux palazzo seneca,0712 567834,isla,IT,06044,via cesare battisti 5
...,...,...,...,...,...,...,...
79,a495,oldcity open-hotel - suites & apartments,,cairo,EG,99999,39 chark compound campus new cairo city
80,a496,live party bed & breakfast,,desenzano del garda,IT,25015,woal cavour 2
81,a497,lala hostala,,chefchaouen,MA,91000,lalla lambda 22
82,a498,orka mountain villa 10,,ovacik,TR,48340,


In [15]:
df_source2["country"] = df_source2.country.apply(lambda x: normalize_country_to_alpha2_code(x))

In [16]:
df_source2

,ID,name,city,country,zip,street,phone
0,ba1,westerns country inn bed & breakfast,johnscity,US,86033,west route 77 447,+1 928-345-4567
1,ba2,natpolin hotel,istanbul,TR,34110,"kanuk ayasofya mahallesi yeğen sok. no:3/1 sultanahmet , fatih, ,",
2,ba3,sleep in cairo hostel,cairo,EG,11511,"11 b, ramses street, downtown",
3,ba4,mekking house,george town,MY,10220,11 lorong carnarvon,
4,ba5,palazzo seneca,isla,IT,6046,"via cesare battisti, 5",+39 0712 567834
...,...,...,...,...,...,...,...
79,ba495,newcity suites & apartments,cairo,EG,11835,23 arban compound infront of new aac campus- new cairo,
80,ba496,bed & breakfast party house,desenzano del garda,IT,25015,via woal verdi 10,+39 44 33 443
81,ba497,hostal la lambada,chefchaouene,MA,91000,rue sidi ahme,+212 44-33445
82,ba498,orka my villa,istanbul,TR,48345,"ovacık mahallesi, istanbul caddesi",


Next we will also normalize all phone numbers to not show the country code by using a Python library called python-phonenumbers (https://github.com/daviddrysdale/python-phonenumbers).

In [17]:
import phonenumbers

In [18]:
def get_national_number(phone_number):
    """
    Given a phone number in any format, return only the part of it that is a national number. 
    This is achieved using phonenumber Python library.
    """
    if phone_number:
        return phonenumbers.parse(phone_number, None).national_number
    else:
        return ""

In [19]:
# get only the national number from source 2
df_source2["phone"] = df_source2.phone.apply(lambda x: get_national_number(x))

In [20]:
df_source2

,ID,name,city,country,zip,street,phone
0,ba1,westerns country inn bed & breakfast,johnscity,US,86033,west route 77 447,9283454567
1,ba2,natpolin hotel,istanbul,TR,34110,"kanuk ayasofya mahallesi yeğen sok. no:3/1 sultanahmet , fatih, ,",
2,ba3,sleep in cairo hostel,cairo,EG,11511,"11 b, ramses street, downtown",
3,ba4,mekking house,george town,MY,10220,11 lorong carnarvon,
4,ba5,palazzo seneca,isla,IT,6046,"via cesare battisti, 5",712567834
...,...,...,...,...,...,...,...
79,ba495,newcity suites & apartments,cairo,EG,11835,23 arban compound infront of new aac campus- new cairo,
80,ba496,bed & breakfast party house,desenzano del garda,IT,25015,via woal verdi 10,4433443
81,ba497,hostal la lambada,chefchaouene,MA,91000,rue sidi ahme,4433445
82,ba498,orka my villa,istanbul,TR,48345,"ovacık mahallesi, istanbul caddesi",


In [21]:
import re
# pre-process the source 1 to have only digits in the number
df_source1["phone"] = df_source1.phone.apply(lambda x: re.sub("[^0-9]", "", str(x)))

In [22]:
df_source1

,ID,name,phone,city,country,zip,street
0,a1,westerns country inn,9283454567,johnscity,US,86033,447 west route 77 az
1,a2,natpolin hostel,,istanbul,TR,34110,kanuk ayasofya mahallesi yeğen sok. no 3/1 sultanahmet fatih
2,a3,sleep in! cairo hostel,,cairo,EG,99999,11 b ramsis st. marouf tower.downtown
3,a4,mekking house,,george town,MY,,11 lorong carnarvon 10220
4,a5,rolando & chateaux palazzo seneca,0712567834,isla,IT,06044,via cesare battisti 5
...,...,...,...,...,...,...,...
79,a495,oldcity open-hotel - suites & apartments,,cairo,EG,99999,39 chark compound campus new cairo city
80,a496,live party bed & breakfast,,desenzano del garda,IT,25015,woal cavour 2
81,a497,lala hostala,,chefchaouen,MA,91000,lalla lambda 22
82,a498,orka mountain villa 10,,ovacik,TR,48340,


In [23]:
# df_source1.to_csv("./data/deep_matcher/s1.csv")
# df_source2.to_csv("./data/deep_matcher/s2.csv")

# Candidate Generation

As discussed in the theoretical part, we don't want to compare each record in source 1 to each record in source 2, and to avoid this we can use various approaches to candidate generation of potential match pairs.

In this demo, we will simply make sure to only compare hotel that are in the same country, this should considerably decrease the amount of comparisons we need to compute. In real-word applications, candidate generation is one of the most important steps of the Entity Matching algorithm. For the purpose of this demo we are only generating candidates based on the country, however, there are many ways this step can be refined. Explore the various options and approaches to candidate generation by following the tutorials provided on the documentation page of the py_entitymatching Python library (http://anhaidgroup.github.io/py_entitymatching/v0.3.x/user_manual/guides.html#stepwise-guides).

In [24]:
import py_entitymatching as em

In [25]:
em.set_key(df_source1, 'ID')
em.set_key(df_source2, 'ID')

True

In [26]:
# instantiate overlap blocker object
ob = em.OverlapBlocker()

#combine two tables from source 1 and source 2 and generate candidate pairs based on overlapping country columns
match_candidate_pairs_df = ob.block_tables(df_source1, df_source2, 'country', 'country', word_level=True, overlap_size=1, 
                    l_output_attrs=['name', 'phone', 'city', 'zip', 'street'], 
                    r_output_attrs=['name', 'phone', 'city', 'zip', 'street'])

/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/py_entitymatching/blocker/overlap_blocker.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_df[l_dummy_overlap_attr] = l_df[l_overlap_attr]
/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/py_entitymatching/blocker/overlap_blocker.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_df[r_dummy_overlap_attr] = r_df[r_overlap_attr]
/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-pack

In [27]:
match_candidate_pairs_df

,_id,ltable_ID,rtable_ID,ltable_name,ltable_phone,ltable_city,ltable_zip,ltable_street,rtable_name,rtable_phone,rtable_city,rtable_zip,rtable_street
0,0,a1,ba1,westerns country inn,9283454567,johnscity,86033,447 west route 77 az,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
1,1,a6,ba1,wake up columbia gateway,4104563367,jessup,20896,8567 washington boulevard md,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
2,2,a14,ba1,vacation inn express hotel & suites monar,4353451467,monar,84182,1511 north highway 176 ut,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
3,3,a15,ba1,night inn by wyndy martinsburg,3042562588,martinsburg,25266,125 viking way wv,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
4,4,a21,ba1,mighty inn & suites mt dora north,3527542200,mount dora,32345,16220 us 431 west fl,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,741,a482,ba498,famia gammon hotel,,faralya,43100,uzunyurt sokak no 77,orka my villa,,istanbul,48345,"ovacık mahallesi, istanbul caddesi"
742,742,a498,ba498,orka mountain villa 10,,ovacik,48340,,orka my villa,,istanbul,48345,"ovacık mahallesi, istanbul caddesi"
743,743,a484,ba499,new inside pool villa,,chalong,83213,58/135 soi anusorn aa 1 chalong,golden triangle resort,44445444,chiang saen,57150,"234 golden triangle, chiang saen"
744,744,a487,ba499,eastwards cozy home,,chiang mai,50993,baan wasin siri maka m.2 t.nong khwai a.hang lank 23/7,golden triangle resort,44445444,chiang saen,57150,"234 golden triangle, chiang saen"


By simply overlapping the two sources by country code we will need to do computaton on more than 746 thousand record pairs. This number grows quadratically with the size of the dataset, which makes selection of a right approach when doing candidate selection so important.

We can filter out a few more record pairs by, for example, allowing only record pairs in which at least one word in the hotel name exists in both pairs.

In [28]:
strict_match_candidate_pairs_df = ob.block_candset(match_candidate_pairs_df, 'name', 'name', word_level=True, overlap_size=1, show_progress=False)

/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/py_entitymatching/blocker/overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[overlap_attr] = values


In [29]:
strict_match_candidate_pairs_df

,_id,ltable_ID,rtable_ID,ltable_name,ltable_phone,ltable_city,ltable_zip,ltable_street,rtable_name,rtable_phone,rtable_city,rtable_zip,rtable_street
0,0,a1,ba1,westerns country inn,9283454567,johnscity,86033,447 west route 77 az,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
2,2,a14,ba1,vacation inn express hotel & suites monar,4353451467,monar,84182,1511 north highway 176 ut,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
3,3,a15,ba1,night inn by wyndy martinsburg,3042562588,martinsburg,25266,125 viking way wv,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
4,4,a21,ba1,mighty inn & suites mt dora north,3527542200,mount dora,32345,16220 us 431 west fl,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
6,6,a50,ba1,east inn karlston,,evanston,82334,1922 harrison drive wy,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447
...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,715,a494,ba494,guest paradise – bedroom home,,surfside beach myrtle beach,29347,"22 north dogwood drive ""paradise""",hostel paradise and home apartments,,myrtle beach,29347,south ocean boulevard 2355
719,719,a495,ba495,oldcity open-hotel - suites & apartments,,cairo,99999,39 chark compound campus new cairo city,newcity suites & apartments,,cairo,11835,23 arban compound infront of new aac campus- new cairo
730,730,a496,ba496,live party bed & breakfast,,desenzano del garda,25015,woal cavour 2,bed & breakfast party house,4433443,desenzano del garda,25015,via woal verdi 10
742,742,a498,ba498,orka mountain villa 10,,ovacik,48340,,orka my villa,,istanbul,48345,"ovacık mahallesi, istanbul caddesi"


Now we have reduced the number of record pairs we need to compare to only around 262 pairs.

# Record Pair Comparison (Threshold-based)

Our next step is to compare each identifier in each potential record pair from the previous step and compute a similarity score for each column comparison between source 1 and source 2.

We can simply do a binary comparison for simple columns like phone and zip code, namely to see if they are a full match or not. For more complex columns like names, street and city, we can use one of the available string comparison algorithms implemented in the textdistance Python library (https://pypi.org/project/textdistance/) or the jellyfish Python library (https://pypi.org/project/jellyfish/). In this particular demo, we will use the Jaro Winkler distance, however you should experiment with various string similarity measures and see which one yields best results for your particular data type.

In [30]:
import textdistance

# add columns to store similarity scores for each record identifier
def binary_comparison(column_value1, column_value2):
    if column_value1 == column_value2:
        return 1
    else:
        return 0
    

def string_similarity_comparison(column_value1, column_value2):
    return textdistance.jaro.normalized_similarity(column_value1, column_value2)

In [31]:
strict_match_candidate_pairs_df["name_sim_score"] = strict_match_candidate_pairs_df.apply(lambda x: string_similarity_comparison(x["ltable_name"], x["rtable_name"]), axis=1)
strict_match_candidate_pairs_df["street_sim_score"] = strict_match_candidate_pairs_df.apply(lambda x: string_similarity_comparison(x["ltable_street"], x["rtable_street"]), axis=1)
strict_match_candidate_pairs_df["city_sim_score"] = strict_match_candidate_pairs_df.apply(lambda x: string_similarity_comparison(x["ltable_city"], x["rtable_city"]), axis=1)
strict_match_candidate_pairs_df["phone_sim_score"] = strict_match_candidate_pairs_df.apply(lambda x: binary_comparison(x["ltable_phone"], x["rtable_phone"]), axis=1)
strict_match_candidate_pairs_df["zip_sim_score"] = strict_match_candidate_pairs_df.apply(lambda x: binary_comparison(x["ltable_zip"], x["rtable_zip"]), axis=1)

/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [32]:
# let's inspect the table with the columns containing similarity score values for each record identifier pair
strict_match_candidate_pairs_df[["ltable_ID", "rtable_ID", "ltable_name", "rtable_name", "name_sim_score", "street_sim_score", "city_sim_score", "phone_sim_score", "zip_sim_score"]]

,ltable_ID,rtable_ID,ltable_name,rtable_name,name_sim_score,street_sim_score,city_sim_score,phone_sim_score,zip_sim_score
0,a1,ba1,westerns country inn,westerns country inn bed & breakfast,0.851852,0.721895,1.000000,0,1
2,a14,ba1,vacation inn express hotel & suites monar,westerns country inn bed & breakfast,0.654697,0.468683,0.540741,0,0
3,a15,ba1,night inn by wyndy martinsburg,westerns country inn bed & breakfast,0.602381,0.450980,0.436027,0,0
4,a21,ba1,mighty inn & suites mt dora north,westerns country inn bed & breakfast,0.619199,0.498529,0.544444,0,0
6,a50,ba1,east inn karlston,westerns country inn bed & breakfast,0.606042,0.481411,0.569444,0,0
...,...,...,...,...,...,...,...,...,...
715,a494,ba494,guest paradise – bedroom home,hostel paradise and home apartments,0.744285,0.521601,0.654321,1,1
719,a495,ba495,oldcity open-hotel - suites & apartments,newcity suites & apartments,0.717142,0.686749,1.000000,1,0
730,a496,ba496,live party bed & breakfast,bed & breakfast party house,0.752735,0.592425,1.000000,0,1
742,a498,ba498,orka mountain villa 10,orka my villa,0.767288,0.000000,0.430556,1,0


Notice that we have given each identifier pair score a range betwee 0 and 1. Actually, the phone number or the zip code matching is hardly as significant as a name or full street address. For this reason, we should boost the scores of each of these columns accordingly. For the purpose of the demo, we are going to use arbitrary number that we selected without any special testing. For the purpose of a production-ready system, the boosting weights should be computed using a more elaborate approach, for example a machine learning approach that defines best boosting values based on the accuracy of the final prediction on the whole gold standard.

In [33]:
# let's boost the scores of the more important fields
strict_match_candidate_pairs_df["name_sim_score"] = strict_match_candidate_pairs_df.name_sim_score.apply(lambda x: x*3)
strict_match_candidate_pairs_df["street_sim_score"] = strict_match_candidate_pairs_df.street_sim_score.apply(lambda x: x*2)

/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
strict_match_candidate_pairs_df.head(5)

,_id,ltable_ID,rtable_ID,ltable_name,ltable_phone,ltable_city,ltable_zip,ltable_street,rtable_name,rtable_phone,rtable_city,rtable_zip,rtable_street,name_sim_score,street_sim_score,city_sim_score,phone_sim_score,zip_sim_score
0,0,a1,ba1,westerns country inn,9283454567,johnscity,86033,447 west route 77 az,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,2.555556,1.443791,1.000000,0,1
2,2,a14,ba1,vacation inn express hotel & suites monar,4353451467,monar,84182,1511 north highway 176 ut,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,1.964092,0.937367,0.540741,0,0
3,3,a15,ba1,night inn by wyndy martinsburg,3042562588,martinsburg,25266,125 viking way wv,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,1.807143,0.901961,0.436027,0,0
4,4,a21,ba1,mighty inn & suites mt dora north,3527542200,mount dora,32345,16220 us 431 west fl,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,1.857598,0.997059,0.544444,0,0
6,6,a50,ba1,east inn karlston,,evanston,82334,1922 harrison drive wy,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,1.818125,0.962821,0.569444,0,0


Next we need to sum up all the similarity scores and set a threshold at which we can regard a record pair an actual match. We can have a maximum score of 8. For the purpose of the demo, we will set the threhold at which we regard something as a match at the score of 5.5. The threshold has to be low enough to account for some records missing data for various columns, but high enough to reach a reasonable level of prediction accuracy. For the production-level solutions, make sure to compute a suitable threshold for your use case by doing additional experimentation on your data.

In [35]:
strict_match_candidate_pairs_df["total_sim_score"] = strict_match_candidate_pairs_df[["name_sim_score", "street_sim_score", "city_sim_score", "phone_sim_score", "zip_sim_score"]].sum(axis=1)

/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
strict_match_candidate_pairs_df.head(5)

,_id,ltable_ID,rtable_ID,ltable_name,ltable_phone,ltable_city,ltable_zip,ltable_street,rtable_name,rtable_phone,rtable_city,rtable_zip,rtable_street,name_sim_score,street_sim_score,city_sim_score,phone_sim_score,zip_sim_score,total_sim_score
0,0,a1,ba1,westerns country inn,9283454567,johnscity,86033,447 west route 77 az,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,2.555556,1.443791,1.000000,0,1,5.999346
2,2,a14,ba1,vacation inn express hotel & suites monar,4353451467,monar,84182,1511 north highway 176 ut,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,1.964092,0.937367,0.540741,0,0,3.442200
3,3,a15,ba1,night inn by wyndy martinsburg,3042562588,martinsburg,25266,125 viking way wv,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,1.807143,0.901961,0.436027,0,0,3.145131
4,4,a21,ba1,mighty inn & suites mt dora north,3527542200,mount dora,32345,16220 us 431 west fl,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,1.857598,0.997059,0.544444,0,0,3.399101
6,6,a50,ba1,east inn karlston,,evanston,82334,1922 harrison drive wy,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,1.818125,0.962821,0.569444,0,0,3.350391


In [37]:
strict_match_candidate_pairs_df = strict_match_candidate_pairs_df[strict_match_candidate_pairs_df["total_sim_score"] > 5.5]

In [38]:
strict_match_candidate_pairs_df.head(5)

,_id,ltable_ID,rtable_ID,ltable_name,ltable_phone,ltable_city,ltable_zip,ltable_street,rtable_name,rtable_phone,rtable_city,rtable_zip,rtable_street,name_sim_score,street_sim_score,city_sim_score,phone_sim_score,zip_sim_score,total_sim_score
0,0,a1,ba1,westerns country inn,9283454567,johnscity,86033,447 west route 77 az,westerns country inn bed & breakfast,9283454567,johnscity,86033,west route 77 447,2.555556,1.443791,1.0,0,1,5.999346
12,12,a2,ba2,natpolin hostel,,istanbul,34110,kanuk ayasofya mahallesi yeğen sok. no 3/1 sultanahmet fatih,natpolin hotel,,istanbul,34110,"kanuk ayasofya mahallesi yeğen sok. no:3/1 sultanahmet , fatih, ,",2.933333,1.859829,1.0,1,1,7.793162
23,23,a3,ba3,sleep in! cairo hostel,,cairo,99999,11 b ramsis st. marouf tower.downtown,sleep in cairo hostel,,cairo,11511,"11 b, ramses street, downtown",2.954545,1.505163,1.0,1,0,6.459709
27,27,a4,ba4,mekking house,,george town,,11 lorong carnarvon 10220,mekking house,,george town,10220,11 lorong carnarvon,3.000000,1.840000,1.0,1,0,6.840000
76,76,a8,ba8,hotel nakoogullari,,batman,72200,ink mah. d.bakır cad. arsan haklari bulvari no 3,hotel nakooğullari,,batman,72200,ink mah. d.bakır cad. arsan haklari bulvari no:3,2.888889,1.972222,1.0,1,1,7.861111


After filtering out all record pairs that have a total similarity score lower than our set threshold, we are left with 442 record pairs. Now we also need to make sure that we do a one to one comparison and only allow the records from source 1 to be matches to one record in source 2.

Let's extract one to one predicted pairs and evaluate the quality based on an annotated gold standard (for the purpose of the demo, the gold standard was not split into training, testing and validation sets, however, this is the recommended way to follow for the developmenet of a production-ready system).

In [39]:
# extract predicted record pairs
predicted_record_pair_list = list()

strict_match_candidate_pairs_df = strict_match_candidate_pairs_df[["ltable_ID", "rtable_ID", "total_sim_score"]]
df_to_list = strict_match_candidate_pairs_df.values.tolist()

In [40]:
# ToDo: redo with pandas: https://stackoverflow.com/questions/39918262/filter-max-values-by-row-and-columns
"""
strict_match_candidate_pairs_df2 = strict_match_candidate_pairs_df.groupby(by=["ltable_ID"])
strict_match_candidate_pairs_df3 = strict_match_candidate_pairs_df2.agg({"total_sim_score":"max"})
strict_match_candidate_pairs_df3.reset_index(inplace=True)
"""

'\nstrict_match_candidate_pairs_df2 = strict_match_candidate_pairs_df.groupby(by=["ltable_ID"])\nstrict_match_candidate_pairs_df3 = strict_match_candidate_pairs_df2.agg({"total_sim_score":"max"})\nstrict_match_candidate_pairs_df3.reset_index(inplace=True)\n'

In [41]:
# extract the unique one to one predictions based on the highest total similarity score between each pair
from collections import defaultdict

value_pair = defaultdict(list)
for x in df_to_list:
    value_pair[x[0]].append([x[1], x[2]])

final_predictions = set()
for x in value_pair:
    final_predictions.add((x, sorted(value_pair[x], key=lambda x: x[1])[-1][0]))

print(len(final_predictions))

52


## Evaluate Prediction Quality (Threshold-based)

In [42]:
from utils import build_annotation_mapping, evaluate_matches

In [43]:
annotation_df = pd.read_csv("./data/gold_standard.csv", low_memory=False)
annotation_df = annotation_df[["ID", "ID2", "match"]].dropna()

In [44]:
annotation_df

,ID,ID2,match
0,a1,ba1,1
1,a2,ba2,1
2,a3,ba3,1
3,a4,ba4,1
4,a5,ba5,1
...,...,...,...
79,a495,ba495,0
80,a496,ba496,0
81,a497,ba497,0
82,a498,ba498,0


In [45]:
gold_standard = build_annotation_mapping(annotation_df)

In [ ]:
recall, precision, f1score = evaluate_matches(final_predictions, gold_standard)

In [47]:
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1score)

Precision: 0.8269230769230769
Recall: 0.7818181818181819
F1-Score: 80.37


Using a simple threhold-based approach we have already achieved a significant level of Precision and Recall without having to rely on a large training corpus of thousands of annotated record pairs. This is one of the main adventages of such simple apporaches. Next we will shortly compare a neural-based Entity Matching system called DeepMatcher.

# Record Pair Comparison (Neural-based, DeepMatcher)

A neural-based approach to Entity Matching has its pros and cons. It still required pre-processing and candidate generations teps, however, the similarity scores we had to compute for each record identifier is now a part of a fully automated process within the neural-based approach, such as DeepMatcher. The drawback of this approach is the fact that often a lot of annotated data with manually matched record pairs is required to be able to train a model that will produce quality match classification results.

In [48]:
import deepmatcher as dm

In [49]:
train, validation, test = dm.data.process(
    path='./data/deep_matcher',
    train='train.csv',
    validation='validation.csv',
    test='test.csv')

/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: MatchingField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)

Reading and processing data from "./data/deep_matcher/train.csv"
/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages

The DeepMatcher framework for neural Entity Matching provides extensive examples on how to use it under its GitHub repository (https://github.com/anhaidgroup/deepmatcher/tree/master/examples). For this experiment we will use a Hybrid DeepMatcher model [citation, page 8]

In [50]:
model = dm.MatchingModel(attr_summarizer=dm.attr_summarizers.Hybrid(word_contextualizer=dm.word_contextualizers.SelfAttention(heads=2)))

In [51]:
model.run_train(
    train,
    validation,
    epochs=5,
    batch_size=8,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=2)

/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: MatchingIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


* Number of trainable parameters: 19590008
===>  TRAIN Epoch 1


/home/ivan/anaconda3/envs/entity_matching_demo/lib/python3.7/site-packages/torch/nn/functional.py:2398: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 1 || Run Time:    1.7 | Load Time:    0.1 || F1:  79.31 | Prec:  74.19 | Rec:  85.19 || Ex/s:  22.19

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.4 | Load Time:    0.0 || F1:  75.68 | Prec:  60.87 | Rec: 100.00 || Ex/s:  52.64

* Best F1: tensor(75.6757, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 4 || Run Time:    1.6 | Load Time:    0.1 || F1:  83.08 | Prec:  71.05 | Rec: 100.00 || Ex/s:  23.10

===>  EVAL Epoch 4
Finished Epoch 4 || Run Time:    0.4 | Load Time:    0.0 || F1:  75.68 | Prec:  60.87 | Rec: 100.00 || Ex/s:  50.24

---------------------

===>  TRAIN Epoch 5


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 5 || Run Time:    1.4 | Load Time:    0.0 || F1:  83.08 | Prec:  71.05 | Rec: 100.00 || Ex/s:  25.49

===>  EVAL Epoch 5
Finished Epoch 5 || Run Time:    0.4 | Load Time:    0.0 || F1:  75.68 | Prec:  60.87 | Rec: 100.00 || Ex/s:  53.28

---------------------

Loading best model...
Training done.


tensor(75.6757, device='cuda:0')

In [52]:
model.run_eval(test)

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.0 || F1:  75.68 | Prec:  60.87 | Rec: 100.00 || Ex/s: 125.78



tensor(75.6757, device='cuda:0')

Note that for the purpose of the demo, we have skipped the candidate generation step for DeepMatcher, hence Recall is at maximum.

DeepMatcher provides multiple tutotials on its usage at https://github.com/anhaidgroup/deepmatcher, continue there for more advanced examples of its usage.